# [Retrieval-based-Voice-Conversion-WebUI](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI) Блокнот для обучения (Адаптировано для Kaggle)

**Версия RVC:** (Укажите версию или коммит, если необходимо для воспроизводимости)

## **Важно: Настройка для Kaggle**

1.  **Добавьте ваши данные и модели:**
    * Загрузите ваш **набор данных** (zip-архив с `.wav` или `.mp3` файлами) как **Kaggle Dataset**.
    * Загрузите **необходимые предварительно обученные модели RVC** (базовые `.pth` для v1/v2, `hubert_base.pt`, `rmvpe.pt`, опционально модели uvr5) также как **Kaggle Dataset**. Рекомендуется создать *отдельный* датасет для моделей для удобства обновления.
2.  **Подключите наборы данных к этому блокноту:**
    * В правой панели Kaggle нажмите "+ Add Data".
    * Найдите и добавьте созданные вами наборы данных (один для аудио, один для моделей).
3.  **Обновите пути в ячейках ниже:**
    * Найдите ячейки с пометкой **"ОТРЕДАКТИРУЙТЕ ЭТИ ПУТИ"**.
    * Замените плейсхолдеры типа `/kaggle/input/your-dataset-name/your_archive.zip` и `/kaggle/input/your-models-dataset-name/` на *реальные* пути к вашим подключенным датасетам.
4.  **Выберите тип GPU:**
    * Убедитесь, что в настройках ноутбука (Accelerator) выбран подходящий GPU (например, T4 x1, P100).

In [ ]:
# @title #Проверить доступность и тип видеокарты
!nvidia-smi

### Установка зависимостей
Устанавливаем необходимые библиотеки. Версии подобраны для совместимости, но если возникнут ошибки при запуске RVC, возможно, потребуется свериться с официальным репозиторием RVC на предмет актуальных версий.

In [ ]:
# @title #Установить зависимости
# Обновляем pip и устанавливаем базовые пакеты
!pip install --upgrade pip setuptools wheel

# Устанавливаем FFmpeg (обычно уже есть в Kaggle, можно закомментировать)
# !apt-get update && apt-get install -y ffmpeg

# Устанавливаем основные Python-зависимости RVC
# Примечание: faiss-cpu используется для широкой совместимости. faiss-gpu быстрее, но сложнее в установке.
!pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118
!pip install faiss-cpu==1.7.4 fairseq gradio==3.14.0 ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2 requests

# Устанавливаем aria2 для скачивания (нужен только если НЕ используете Kaggle Dataset для моделей)
!apt-get -y install -qq aria2 || echo "aria2 уже установлен или ошибка установки, продолжаем..."

### Клонирование репозитория RVC
Клонируем официальный репозиторий RVC в рабочую директорию Kaggle `/kaggle/working/`.

In [ ]:
# @title #Клонировать репозиторий
import os

# Переходим в рабочую директорию Kaggle
WORK_DIR = '/kaggle/working/'
if os.getcwd() != WORK_DIR:
  os.chdir(WORK_DIR)

# Удаляем папку, если она уже существует (для чистого клонирования при перезапуске)
!rm -rf Retrieval-based-Voice-Conversion-WebUI

# Клонируем репозиторий
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git

# Переходим в папку репозитория
RVC_DIR = os.path.join(WORK_DIR, 'Retrieval-based-Voice-Conversion-WebUI')
os.chdir(RVC_DIR)

# (Опционально) Переключиться на конкретный коммит для стабильности
# !git checkout КОНКРЕТНЫЙ_КОММИТ_ХЭШ

print(f"Текущая директория: {os.getcwd()}")

### Подготовка необходимых моделей

**Рекомендуемый способ:** Использовать **Kaggle Dataset**, содержащий все нужные файлы (`.pth`, `.pt`, `.index` и т.д.). Это значительно ускоряет запуск и обеспечивает стабильность.

**Альтернативный способ:** Если вы НЕ загрузили модели как Kaggle Dataset, можно раскомментировать и выполнить ячейки ниже для скачивания моделей с Hugging Face с помощью `aria2c`. **Это будет медленнее.**

In [ ]:
# @title #✅ Копировать модели из Kaggle Dataset (Рекомендуемый способ)
import os
import shutil
import glob

# --- ⛔ ОТРЕДАКТИРУЙТЕ ЭТОТ ПУТЬ ⛔ ---
# Укажите путь к вашему Kaggle Dataset, КУДА вы загрузили модели.
# Пример: '/kaggle/input/rvc-essential-models/'
KAGGLE_MODELS_PATH = "/kaggle/input/your-models-dataset-name/" # @param {type:"string"}
# --- КОНЕЦ РЕДАКТИРОВАНИЯ ---

# Проверяем, существует ли указанный путь
if not os.path.exists(KAGGLE_MODELS_PATH):
    print(f"❌ ОШИБКА: Директория {KAGGLE_MODELS_PATH} не найдена. Убедитесь, что вы правильно указали путь и подключили датасет с моделями.")
else:
    print(f"Найден датасет с моделями: {KAGGLE_MODELS_PATH}")
    # Целевые директории в клонированном репозитории RVC
    repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
    pretrained_v1_dest = os.path.join(repo_dir, 'pretrained')
    pretrained_v2_dest = os.path.join(repo_dir, 'pretrained_v2')
    uvr5_dest = os.path.join(repo_dir, 'uvr5_weights')
    hubert_dest = repo_dir
    rmvpe_dest = repo_dir

    # Создаем целевые директории, если они не существуют
    os.makedirs(pretrained_v1_dest, exist_ok=True)
    os.makedirs(pretrained_v2_dest, exist_ok=True)
    os.makedirs(uvr5_dest, exist_ok=True)

    # Функция для безопасного копирования файла
    def safe_copy_file(src_path, dst_dir_or_path):
        if not os.path.exists(src_path):
            print(f"⚠️ Файл не найден (пропускаем): {src_path}")
            return False
        try:
            shutil.copy(src_path, dst_dir_or_path)
            print(f"Скопировано: {src_path} -> {dst_dir_or_path}")
            return True
        except Exception as e:
            print(f"❌ Ошибка копирования {src_path} в {dst_dir_or_path}: {e}")
            return False

    # Функция для копирования всех файлов из папки
    def copy_folder_contents(src_folder, dst_folder):
        if not os.path.exists(src_folder):
            print(f"⚠️ Директория не найдена (пропускаем): {src_folder}")
            return
        print(f"Копирование содержимого из {src_folder} в {dst_folder}...")
        for item_name in os.listdir(src_folder):
            s = os.path.join(src_folder, item_name)
            d = os.path.join(dst_folder, item_name)
            if os.path.isfile(s):
                safe_copy_file(s, d)
            # Можно добавить копирование подпапок, если нужно

    # --- Выполнение копирования ---
    print("\nНачинаем копирование моделей...")

    # Копируем базовые модели (v1 и v2) - Предполагаем, что они лежат в папках 'pretrained' и 'pretrained_v2' в вашем датасете
    copy_folder_contents(os.path.join(KAGGLE_MODELS_PATH, 'pretrained'), pretrained_v1_dest)
    copy_folder_contents(os.path.join(KAGGLE_MODELS_PATH, 'pretrained_v2'), pretrained_v2_dest)

    # Копируем модели UVR5 (Предполагаем, что они лежат в папке 'uvr5_weights')
    copy_folder_contents(os.path.join(KAGGLE_MODELS_PATH, 'uvr5_weights'), uvr5_dest)

    # Копируем Hubert и RMVPE (Предполагаем, что они лежат в корне вашего датасета моделей)
    safe_copy_file(os.path.join(KAGGLE_MODELS_PATH, 'hubert_base.pt'), hubert_dest)
    safe_copy_file(os.path.join(KAGGLE_MODELS_PATH, 'rmvpe.pt'), rmvpe_dest)

    # (Опционально) Копируем другие файлы, если они есть в датасете
    # safe_copy_file(os.path.join(KAGGLE_MODELS_PATH, 'другой_файл.pth'), repo_dir)

    print("\n✅ Копирование моделей из Kaggle Dataset завершено.")
    print("Проверьте наличие файлов в папках 'pretrained', 'pretrained_v2', 'uvr5_weights' и в корне репозитория.")
    # !ls -l {pretrained_v1_dest}
    # !ls -l {pretrained_v2_dest}
    # !ls -l {uvr5_dest}
    # !ls -l {hubert_dest}/hubert_base.pt
    # !ls -l {rmvpe_dest}/rmvpe.pt


In [ ]:
# @title #⬇️ Скачать базовые модели (Альтернатива: если НЕ используете Kaggle Dataset)
# @markdown #Если вы скопировали модели из Kaggle Dataset на предыдущем шаге, ПРОПУСТИТЕ эту ячейку.
import os
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
 os.chdir(repo_dir)
print(f"Текущая директория: {os.getcwd()}")

# Создаем директории
!mkdir -p pretrained pretrained_v2

print("Скачивание базовых моделей v1...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d pretrained -o f0G40k.pth

print("Скачивание базовых моделей v2...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D40k.pth -d pretrained_v2 -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G40k.pth -d pretrained_v2 -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D40k.pth -d pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G40k.pth -d pretrained_v2 -o f0G40k.pth
print("Скачивание базовых моделей завершено.")

In [ ]:
# @title #⬇️ Скачать модели для разделения вокала (Альтернатива: если НЕ используете Kaggle Dataset)
# @markdown #Если вы скопировали модели из Kaggle Dataset, ПРОПУСТИТЕ эту ячейку.
import os
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
 os.chdir(repo_dir)

!mkdir -p uvr5_weights
print("Скачивание моделей UVR5...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d uvr5_weights -o HP2-Вокал+Инструментал.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d uvr5_weights -o HP5-Основной_вокал+Другие_инструменты.pth
print("Скачивание UVR5 завершено.")

In [ ]:
# @title #⬇️ Скачать hubert_base (Альтернатива: если НЕ используете Kaggle Dataset)
# @markdown #Если вы скопировали модели из Kaggle Dataset, ПРОПУСТИТЕ эту ячейку.
import os
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
 os.chdir(repo_dir)

print("Скачивание hubert_base.pt...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d . -o hubert_base.pt
print("Скачивание hubert_base.pt завершено.")

In [ ]:
# @title #⬇️ Скачать rmvpe модель (Альтернатива: если НЕ используете Kaggle Dataset)
# @markdown #Если вы скопировали модели из Kaggle Dataset, ПРОПУСТИТЕ эту ячейку.
import os
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
 os.chdir(repo_dir)

print("Скачивание rmvpe.pt...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -d . -o rmvpe.pt
print("Скачивание rmvpe.pt завершено.")

### Подготовка набора данных для обучения
Распаковываем ваш аудио-датасет (zip-архив) из Kaggle Datasets в рабочую директорию.

In [ ]:
# @title #Распаковать набор данных из Kaggle Dataset
import os
import zipfile

# --- ⛔ ОТРЕДАКТИРУЙТЕ ЭТОТ ПУТЬ ⛔ ---
# Укажите путь к вашему zip-архиву с аудио-данными в Kaggle Datasets
# Пример: /kaggle/input/my-voice-dataset/my_audio.zip
DATASET_ZIP_PATH = "/kaggle/input/your-dataset-name/your_dataset_archive.zip"  # @param {type:"string"}
# --- КОНЕЦ РЕДАКТИРОВАНИЯ ---

# Имя папки, куда будут распакованы аудиофайлы
DATASET_FOLDER_NAME = "dataset" # @param {type:"string"}
DATASET_EXTRACT_DIR = os.path.join("/kaggle/working/", DATASET_FOLDER_NAME)

# Создаем директорию для распаковки
os.makedirs(DATASET_EXTRACT_DIR, exist_ok=True)

# Распаковываем архив
print(f"Попытка распаковки {DATASET_ZIP_PATH} в {DATASET_EXTRACT_DIR}...")
try:
    if not os.path.exists(DATASET_ZIP_PATH):
         print(f"❌ ОШИБКА: Архив не найден по пути {DATASET_ZIP_PATH}. Убедитесь, что путь указан верно и Dataset подключен.")
    else:
        with zipfile.ZipFile(DATASET_ZIP_PATH, 'r') as zip_ref:
            zip_ref.extractall(DATASET_EXTRACT_DIR)
        print(f"✅ Набор данных успешно распакован в {DATASET_EXTRACT_DIR}")
        # Опционально: Посмотреть содержимое
        print("\nСодержимое папки с датасетом:")
        !ls -lh "{DATASET_EXTRACT_DIR}" | head -n 10 # Показываем первые 10 файлов/папок
except zipfile.BadZipFile:
    print(f"❌ ОШИБКА: Файл {DATASET_ZIP_PATH} не является zip-архивом или поврежден.")
except Exception as e:
    print(f"❌ ОШИБКА при распаковке архива: {e}")

# @title # (Опционально) Переименовать дубликаты файлов (с '~' в конце)
# @markdown # Эта команда ищет файлы с тильдой (`~`) в конце имени, которые могут создаваться в некоторых ОС при копировании файлов с одинаковыми именами.
# @markdown # Она пытается переименовать их в формат 'имя_файла_ЦИФРА.расширение'.
# @markdown # ВНИМАНИЕ: Использует команду `rename` (Perl). Если она не работает, возможно, потребуется другая команда или Python-скрипт.
RENAME_DUPLICATES = False # @param {type:"boolean"}

if RENAME_DUPLICATES and os.path.exists(DATASET_EXTRACT_DIR):
    print("\nИщем и переименовываем файлы с '~' в конце...")
    # Проверяем содержимое перед переименованием
    print("Содержимое до переименования (файлы с '~'):")
    !find "{DATASET_EXTRACT_DIR}/" -name '*.*~*'

    # Выполняем переименование (используем find для рекурсивного поиска и Perl rename)
    # Если эта команда вызывает ошибку, возможно, утилита rename имеет другой синтаксис
    !find "{DATASET_EXTRACT_DIR}/" -name '*.*~*' -exec rename 's/(\.\w+)~(\d*)/_$2$1/' '{}' \; || echo "Команда rename не удалась или не найдено файлов для переименования."

    # Проверяем содержимое после переименования
    print("\nСодержимое после переименования (проверка):")
    !ls -a "{DATASET_EXTRACT_DIR}/"
else:
    print("\nПропуск переименования дубликатов.")

### Запуск Web UI для обучения и инференса
Запускаем веб-интерфейс Gradio. Он позволяет удобно выполнять все шаги: предварительную обработку, извлечение признаков, обучение и тестирование модели.

**ВНИМАНИЕ:** В Kaggle прямой доступ к веб-интерфейсу через публичную ссылку Gradio (`share=True`) **часто не работает** из-за сетевых ограничений Kaggle.
* Если ссылка `*.gradio.live` не открывается или работает нестабильно, вам может потребоваться использовать **сервис туннелирования**, например `ngrok`.
* Использование `ngrok` требует отдельной регистрации, получения токена и запуска туннеля (это не входит в данный ноутбук, но инструкции легко найти в интернете).

In [ ]:
# @title #▶️ Запустить webui
import os
# Убедимся, что мы в директории репозитория RVC
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
  print(f"Переходим в директорию {repo_dir}...")
  os.chdir(repo_dir)

# %load_ext tensorboard
# %tensorboard --logdir ./logs --port 6006 # Tensorboard может потребовать настройки порта или туннелирования в Kaggle

print("Запускаем infer-web.py...")
print("Пожалуйста, дождитесь появления ссылки *.gradio.live или ошибки.")
print("Если ссылка не работает, возможно, потребуется ngrok.")

# Запускаем webui. Флаг --share пытается создать публичную ссылку Gradio.
# Удаляем флаг --colab, так как мы в Kaggle.
!python infer-web.py --pycmd python3 --share

### Сохранение результатов
После завершения обучения ваши файлы модели (`.pth`, `.index`, `.npy`) будут находиться:
* Чекпоинты обучения (`G_*.pth`, `D_*.pth`, `added_*.index`, `total_*.npy`): в подпапке `./logs/ВАШЕ_ИМЯ_МОДЕЛИ/`
* Финальный файл модели для инференса (`ВАШЕ_ИМЯ_МОДЕЛИ.pth`): в папке `./weights/`

**Как сохранить файлы из Kaggle:**
1.  **Остановите выполнение** ячейки с Web UI (нажмите кнопку Stop).
2.  **(Рекомендуется)** Выполните следующую ячейку кода "Копирование важных файлов...", чтобы скопировать нужные файлы в одну папку для удобства.
3.  Нажмите кнопку **"Save Version"** (вверху справа).
4.  Выберите тип "Save & Run All (Commit)" и нажмите **"Save"**.
5.  Дождитесь завершения работы ноутбука (это может занять время).
6.  После завершения перейдите на страницу вашей сохраненной версии (Version).
7.  В разделе **"Output"** -> **"/kaggle/working/"** вы найдете все файлы, созданные во время сессии, включая скопированные модели (например, в папке `saved_model`). Вы сможете скачать их оттуда.

In [ ]:
# @title #💾 Копирование важных файлов модели в /kaggle/working/saved_model для удобства сохранения
# @markdown #Отредактируйте имя и эпоху модели, которую хотите сохранить.
import os
import shutil
import glob

# --- ⛔ ОТРЕДАКТИРУЙТЕ ПАРАМЕТРЫ МОДЕЛИ ⛔ ---
# Имя модели (должно совпадать с именем папки в logs/)
MODELNAME = "my_voice"  # @param {type:"string"}
# Эпоха модели для сохранения чекпоинтов G/D (например, последняя сохраненная)
# Укажите 0, если хотите пропустить копирование чекпоинтов G/D.
MODELEPOCH = 500  # @param {type:"integer"}
# Искать ли самый последний файл .index? Если False, скопирует ВСЕ .index файлы.
FIND_LATEST_INDEX = True # @param {type:"boolean"}
# --- КОНЕЦ РЕДАКТИРОВАНИЯ ---

# Пути
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
logs_dir = os.path.join(repo_dir, 'logs', MODELNAME)
weights_dir = os.path.join(repo_dir, 'weights')
output_dir = '/kaggle/working/saved_model'

# Создаем папку для сохранения в /kaggle/working/
os.makedirs(output_dir, exist_ok=True)

# --- Имена файлов ---
final_weights_name = f'{MODELNAME}.pth'
index_pattern = 'added_*.index' # Шаблон для поиска index файлов
npy_pattern = 'total_*.npy'     # Шаблон для поиска npy файлов

# Функция для безопасного копирования
def safe_copy(src, dst):
    try:
        if os.path.exists(src):
            shutil.copy(src, dst)
            print(f'Скопировано: {src} -> {dst}')
            return True
        else:
            print(f'⚠️ Файл не найден (пропускаем): {src}')
            return False
    except Exception as e:
        print(f'❌ Ошибка копирования {src} в {dst}: {e}')
        return False

# --- Выполнение копирования ---
print(f"Копирование файлов модели '{MODELNAME}' в {output_dir}...")

# 1. Копируем финальный .pth файл из папки weights
final_weights_path = os.path.join(weights_dir, final_weights_name)
safe_copy(final_weights_path, output_dir)

# 2. Копируем чекпоинты G и D указанной эпохи (если эпоха > 0)
if MODELEPOCH > 0:
    g_pth_name = f'G_{MODELEPOCH}.pth'
    d_pth_name = f'D_{MODELEPOCH}.pth'
    safe_copy(os.path.join(logs_dir, g_pth_name), output_dir)
    safe_copy(os.path.join(logs_dir, d_pth_name), output_dir)
else:
    print("Пропуск копирования чекпоинтов G/D (эпоха = 0).")

# 3. Копируем .index файлы из папки logs/MODELNAME
index_files = sorted(glob.glob(os.path.join(logs_dir, index_pattern)), key=os.path.getmtime)
if index_files:
    if FIND_LATEST_INDEX:
        latest_index_file = index_files[-1]
        print(f"Найден последний index файл: {latest_index_file}")
        safe_copy(latest_index_file, output_dir)
    else:
        print(f"Найдены index файлы: {index_files}. Копируем все...")
        for file_path in index_files:
            safe_copy(file_path, output_dir)
else:
    print(f"⚠️ Файлы .index по шаблону '{index_pattern}' не найдены в {logs_dir}")

# 4. Копируем .npy файл(ы) признаков из папки logs/MODELNAME
npy_files = glob.glob(os.path.join(logs_dir, npy_pattern))
if npy_files:
    print(f"Найдены npy файлы: {npy_files}. Копируем...")
    for file_path in npy_files:
        safe_copy(file_path, output_dir)
else:
    print(f"⚠️ Файлы .npy по шаблону '{npy_pattern}' не найдены в {logs_dir}")

print("\n✅ Копирование завершено. Проверьте содержимое папки 'saved_model':")
!ls -lh /kaggle/working/saved_model/

### Дополнительные команды (ручной запуск этапов)
Следующие ячейки позволяют выполнить шаги предварительной обработки, извлечения признаков и обучения вручную, без использования Web UI. Это может быть полезно для отладки или если Web UI не запускается.

In [ ]:
# @title #🛠️ Ручная предварительная обработка (Альтернатива WebUI)
import os
# Убедимся, что мы в директории репозитория RVC
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
  print(f"Переходим в директорию {repo_dir}...")
  os.chdir(repo_dir)

# @markdown --- Параметры --- 
# @markdown Имя модели (для папки логов)
MODELNAME = "my_voice"  # @param {type:"string"}
# @markdown Частота дискретизации (должна совпадать с целевой моделью, например 40k)
BITRATE = "40k" # @param ["32k", "40k", "48k"]
# @markdown Количество потоков (CPU ядер)
THREADCOUNT = 2  # @param {type:"integer"}
# @markdown Путь к папке с вашими аудиофайлами (распакованными ранее)
DATASET_DIR = "/kaggle/working/dataset" # @param {type:"string"}

# Формируем числовое значение частоты дискретизации
sr_map = {'32k': 32000, '40k': 40000, '48k': 48000}
SAMPLE_RATE = sr_map.get(BITRATE, 40000)

print(f"Запуск предварительной обработки для модели '{MODELNAME}'...")
print(f"Исходные данные: {DATASET_DIR}")
print(f"Частота дискретизации: {SAMPLE_RATE} ({BITRATE})")
print(f"Потоков: {THREADCOUNT}")
print(f"Логи будут сохранены в: logs/{MODELNAME}")

# Важно: Экранируем путь и имя модели, если они содержат пробелы или спецсимволы (хотя рекомендуется избегать)
# Используем одинарные кавычки внутри f-строки для строк Python, которые затем станут частью команды shell
DATASET_DIR_ESCAPED = f'\"{DATASET_DIR}\"'
MODELNAME_ESCAPED = f'\"{MODELNAME}\"'

# Собираем команду как строку Python
command = f"!python trainset_preprocess_pipeline_print.py {DATASET_DIR_ESCAPED} {SAMPLE_RATE} {THREADCOUNT} logs/{MODELNAME_ESCAPED} True"
print(f"Executing command: {command}")

# Запускаем команду (эта строка кода Python будет представлена как строка JSON)
!python trainset_preprocess_pipeline_print.py "{DATASET_DIR}" {SAMPLE_RATE} {THREADCOUNT} logs/"{MODELNAME}" True

In [ ]:
# @title #🛠️ Ручное извлечение признаков (Альтернатива WebUI)
import os
# Убедимся, что мы в директории репозитория RVC
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
  print(f"Переходим в директорию {repo_dir}...")
  os.chdir(repo_dir)

# @markdown --- Параметры --- 
# @markdown Имя модели (из предыдущего шага)
MODELNAME = "my_voice"  # @param {type:"string"}
# @markdown Количество потоков (CPU ядер)
THREADCOUNT = 2  # @param {type:"integer"}
# @markdown Алгоритм извлечения высоты тона (F0)
ALGO = "rmvpe"  # @param ["pm", "harvest", "crepe", "rmvpe"]
# @markdown Версия RVC (для извлечения признаков Hubert)
RVC_VERSION = "v2" # @param ["v1", "v2"]
# @markdown Использовать half-precision (fp16)? (Экономит память GPU)
USE_HALF_PRECISION = True # @param {type:"boolean"}

print(f"Запуск извлечения признаков для модели '{MODELNAME}'...")

# Шаг 1: Извлечение F0 (высоты тона)
print(f"\nШаг 1: Извлечение F0 с использованием '{ALGO}'...")
# Экранируем имя модели для команды shell
MODELNAME_ESCAPED = f'\"{MODELNAME}\"'
LOGS_DIR = f'logs/{MODELNAME_ESCAPED}' # Путь к логам модели
# Собираем команду как строку Python
f0_command = f"!python extract_f0_print.py {LOGS_DIR} {THREADCOUNT} {ALGO}"
print(f"Executing command: {f0_command}")
# Запускаем команду
!python extract_f0_print.py logs/"{MODELNAME}" {THREADCOUNT} {ALGO}

# Шаг 2: Извлечение признаков Hubert
print(f"\nШаг 2: Извлечение признаков Hubert (версия {RVC_VERSION})...")
# Параметры: устройство (cuda:0), кол-во GPU (1), ранг GPU (0), порт (0 - не используется), директория логов, версия RVC, использовать ли half-precision
# Собираем команду как строку Python
feature_command = f"!python extract_feature_print.py cuda:0 1 0 0 {LOGS_DIR} {RVC_VERSION} {USE_HALF_PRECISION}"
print(f"Executing command: {feature_command}")
# Запускаем команду
!python extract_feature_print.py cuda:0 1 0 0 logs/"{MODELNAME}" {RVC_VERSION} {USE_HALF_PRECISION}

In [ ]:
# @title #🛠️ Ручное обучение модели (Альтернатива WebUI)
import os
# Убедимся, что мы в директории репозитория RVC
repo_dir = '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI'
if os.getcwd() != repo_dir:
  print(f"Переходим в директорию {repo_dir}...")
  os.chdir(repo_dir)

# @markdown --- Параметры Обучения --- 
# @markdown Имя модели (эксперимента)
MODELNAME = "my_voice"  # @param {type:"string"}
# @markdown Частота дискретизации модели (должна совпадать с предобработкой)
MODELSAMPLE = "40k"  # @param ["32k", "40k", "48k"]
# @markdown Версия RVC (должна совпадать с извлечением признаков)
RVC_VERSION = "v2" # @param ["v1", "v2"]
# @markdown Использовался ли предварительный расчет F0? (1=да, 0=нет)
PRE_F0 = 1 # @param [0, 1]
# @markdown Размер пакета (Batch Size) - подберите под VRAM вашей GPU (уменьшайте при ошибках CUDA out of memory)
BATCHSIZE = 8  # @param {type:"integer"}
# @markdown Номер используемого GPU (обычно 0 в Kaggle)
USEGPU = "0"  # @param {type:"string"}
# @markdown Общее количество эпох обучения
TOTAL_EPOCH = 100  # @param {type:"integer"}
# @markdown Интервал сохранения модели (в эпохах)
SAVE_EPOCH = 10  # @param {type:"integer"}
# @markdown Кэшировать ли обучающий набор данных в RAM? (1=да, 0=нет) - Ускоряет обучение, но требует больше RAM.
CACHE_DATA = 1  # @param [0, 1]
# @markdown Сохранять ли только последний чекпоинт и оптимизатор? (1=да, 0=нет) - Экономит место.
ONLY_LATEST = 0  # @param [0, 1]

# --- Определение путей к предварительно обученным моделям --- 
# Определяем, используется ли F0
f0_prefix = "f0" if PRE_F0 == 1 else ""
# Формируем пути
pretrained_base = f'pretrained_{RVC_VERSION}'
pretrained_g = os.path.join(pretrained_base, f'{f0_prefix}G{MODELSAMPLE}.pth')
pretrained_d = os.path.join(pretrained_base, f'{f0_prefix}D{MODELSAMPLE}.pth')

# Проверяем наличие файлов pretrain
error_message = ""
if not os.path.exists(pretrained_g):
    error_message += f"\n❌ ОШИБКА: Файл pretrain G не найден: {pretrained_g}. "
if not os.path.exists(pretrained_d):
    error_message += f"\n❌ ОШИБКА: Файл pretrain D не найден: {pretrained_d}. "
if error_message:
    print(error_message + "Убедитесь, что базовые модели {RVC_VERSION} были скачаны или скопированы.")
else:
    print(f"Запуск обучения модели '{MODELNAME}'...")
    print(f"Версия RVC: {RVC_VERSION}, Частота: {MODELSAMPLE}, F0: {'Да' if PRE_F0 == 1 else 'Нет'}")
    print(f"Batch Size: {BATCHSIZE}, Эпох: {TOTAL_EPOCH}, Сохранение каждые: {SAVE_EPOCH} эпох")
    print(f"Pretrain G: {pretrained_g}")
    print(f"Pretrain D: {pretrained_d}")

    # --- Сборка команды Shell --- 
    # Экранируем имя модели на всякий случай
    MODELNAME_ESCAPED = f'\"{MODELNAME}\"'

    # Собираем команду как ОДНУ строку для выполнения в shell
    # Используем f-строки для подстановки переменных Python
    # Используем \ для переноса строки внутри команды shell (необязательно, но улучшает читаемость в выводе)
    train_command_str = (
        f"python train_nsf_sim_cache_sid_load_pretrain.py \\\
"
        f"-e {MODELNAME_ESCAPED} \\\
"
        f"-sr {MODELSAMPLE} \\\
"
        f"-f0 {PRE_F0} \\\
"
        f"-bs {BATCHSIZE} \\\
"
        f"-g {USEGPU} \\\
"
        f"-te {TOTAL_EPOCH} \\\
"
        f"-se {SAVE_EPOCH} \\\
"
        f"-pg {pretrained_g} \\\
"
        f"-pd {pretrained_d} \\\
"
        f"-l {ONLY_LATEST} \\\
"
        f"-c {CACHE_DATA} \\\
"
        f"-v {RVC_VERSION}"
    )

    print("\nКоманда для запуска:")
    print(train_command_str)

    # Запускаем обучение с помощью ! оператора IPython
    # Переменная train_command_str будет подставлена сюда
    !{train_command_str}
